[View in Colaboratory](https://colab.research.google.com/github/brucecmd/learn_gluon/blob/master/multi_layer_perceptron.ipynb)

In [0]:
from mxnet import nd, autograd
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet import gluon
import random

In [0]:
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)

In [0]:
batch_size = 256
transformer = gdata.vision.transforms.ToTensor()
train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True)
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=True)

In [0]:
num_inputs = 784
num_hidden = 256
num_outputs = 10
w1 = nd.random_normal(0,1,shape=(num_inputs, num_hidden))
b1 = nd.zeros(num_hidden)
w2 = nd.random_normal(0,1,shape=(num_hidden, num_outputs))
b2 = nd.zeros(num_outputs)
params = [w1, b1, w2, b2]
for p in params:
    p.attach_grad()

In [0]:
num_inputs = 28 * 28
def net(x):
    return nd.dot(nd.relu(nd.dot(x.reshape(-1,num_inputs),w1)+b1),w2) + b2

In [0]:
loss_func = gloss.SoftmaxCrossEntropyLoss()

In [0]:
def sgd(params, batch_size, lr):
    for p in params:
        p[:] -= p.grad * lr / batch_size

In [0]:
def accuracy(y, y_hat):
    return (y_hat.argmax(axis=1)==y.astype('float32')).mean().asscalar()

In [0]:
def estimate_accuracy(data_iter, net):
    total_acc = 0
    for data, label in data_iter:
        y_hat = net(data)
        acc = accuracy(label, y_hat)
        total_acc += acc
    return total_acc / len(data_iter)

In [57]:
epochs = 10
lr = 0.01
batch_size = 256
for i in range(epochs):
    for data, label in train_iter:
        with autograd.record():
            y_hat = net(data)
            l = loss_func(y_hat,label)
        l.backward()
        sgd(params, batch_size, lr)
    train_acc = estimate_accuracy(train_iter, net)
    test_acc = estimate_accuracy(test_iter, net)
    print('epochs [%d], train acc[%f], test acc[%f]'%(i, train_acc, test_acc))

epochs [0], train acc[0.672185], test acc[0.677930]
epochs [1], train acc[0.719515], test acc[0.724316]
epochs [2], train acc[0.737916], test acc[0.736035]
epochs [3], train acc[0.752405], test acc[0.754199]
epochs [4], train acc[0.761458], test acc[0.760059]
epochs [5], train acc[0.767503], test acc[0.769434]
epochs [6], train acc[0.770180], test acc[0.768066]
epochs [7], train acc[0.777870], test acc[0.773047]
epochs [8], train acc[0.776230], test acc[0.776270]
epochs [9], train acc[0.784087], test acc[0.779980]
